# Exercise. Using APIs:
We are going to use [teleport api](https://developers.teleport.org/api/), which is an api that among other things provide country and city information.

Using teleport's API, answer the question:

**Which country in Europe is the best one for an Architect in terms of salaries (50% percentile)?**

Make sure you check the API Explorer to find all of the required endpoints.



In [1]:
import requests
BASE_URL = "https://api.teleport.org/api/"
continents = requests.get(BASE_URL + "continents/").json()

In [2]:
continents

{'_links': {'continent:items': [{'href': 'https://api.teleport.org/api/continents/geonames:AF/',
    'name': 'Africa'},
   {'href': 'https://api.teleport.org/api/continents/geonames:AN/',
    'name': 'Antarctica'},
   {'href': 'https://api.teleport.org/api/continents/geonames:AS/',
    'name': 'Asia'},
   {'href': 'https://api.teleport.org/api/continents/geonames:EU/',
    'name': 'Europe'},
   {'href': 'https://api.teleport.org/api/continents/geonames:NA/',
    'name': 'North America'},
   {'href': 'https://api.teleport.org/api/continents/geonames:OC/',
    'name': 'Oceania'},
   {'href': 'https://api.teleport.org/api/continents/geonames:SA/',
    'name': 'South America'}],
  'curies': [{'href': 'https://developers.teleport.org/api/resources/Location/#!/relations/{rel}/',
    'name': 'location',
    'templated': True},
   {'href': 'https://developers.teleport.org/api/resources/City/#!/relations/{rel}/',
    'name': 'city',
    'templated': True},
   {'href': 'https://developers.telepo

In [3]:
[c for c in continents["_links"]["continent:items"] if c["name"] == "Europe"]

[{'href': 'https://api.teleport.org/api/continents/geonames:EU/',
  'name': 'Europe'}]

In [4]:
europe_countries = requests.get(BASE_URL + "/continents/geonames:EU/countries/").json()

In [5]:
europe_countries["_links"]["country:items"]

[{'href': 'https://api.teleport.org/api/countries/iso_alpha2:AX/',
  'name': 'Aland Islands'},
 {'href': 'https://api.teleport.org/api/countries/iso_alpha2:AL/',
  'name': 'Albania'},
 {'href': 'https://api.teleport.org/api/countries/iso_alpha2:AD/',
  'name': 'Andorra'},
 {'href': 'https://api.teleport.org/api/countries/iso_alpha2:AT/',
  'name': 'Austria'},
 {'href': 'https://api.teleport.org/api/countries/iso_alpha2:BY/',
  'name': 'Belarus'},
 {'href': 'https://api.teleport.org/api/countries/iso_alpha2:BE/',
  'name': 'Belgium'},
 {'href': 'https://api.teleport.org/api/countries/iso_alpha2:BA/',
  'name': 'Bosnia and Herzegovina'},
 {'href': 'https://api.teleport.org/api/countries/iso_alpha2:BG/',
  'name': 'Bulgaria'},
 {'href': 'https://api.teleport.org/api/countries/iso_alpha2:HR/',
  'name': 'Croatia'},
 {'href': 'https://api.teleport.org/api/countries/iso_alpha2:CY/',
  'name': 'Cyprus'},
 {'href': 'https://api.teleport.org/api/countries/iso_alpha2:CZ/',
  'name': 'Czechia'},


In [6]:
from glom import glom
countries_dict = dict(zip(
    glom(europe_countries, ("_links.country:items", ["name"])),
    glom(europe_countries, ("_links.country:items", ["href"]))
))

In [7]:
countries_dict

{'Aland Islands': 'https://api.teleport.org/api/countries/iso_alpha2:AX/',
 'Albania': 'https://api.teleport.org/api/countries/iso_alpha2:AL/',
 'Andorra': 'https://api.teleport.org/api/countries/iso_alpha2:AD/',
 'Austria': 'https://api.teleport.org/api/countries/iso_alpha2:AT/',
 'Belarus': 'https://api.teleport.org/api/countries/iso_alpha2:BY/',
 'Belgium': 'https://api.teleport.org/api/countries/iso_alpha2:BE/',
 'Bosnia and Herzegovina': 'https://api.teleport.org/api/countries/iso_alpha2:BA/',
 'Bulgaria': 'https://api.teleport.org/api/countries/iso_alpha2:BG/',
 'Croatia': 'https://api.teleport.org/api/countries/iso_alpha2:HR/',
 'Cyprus': 'https://api.teleport.org/api/countries/iso_alpha2:CY/',
 'Czechia': 'https://api.teleport.org/api/countries/iso_alpha2:CZ/',
 'Denmark': 'https://api.teleport.org/api/countries/iso_alpha2:DK/',
 'Estonia': 'https://api.teleport.org/api/countries/iso_alpha2:EE/',
 'Faroe Islands': 'https://api.teleport.org/api/countries/iso_alpha2:FO/',
 'Finla

In [8]:
portugal_data = requests.get(
                        "https://api.teleport.org/api/countries/iso_alpha2:PT/salaries"
).json()

In [9]:
portugal_data.keys()

dict_keys(['_links', 'salaries'])

In [12]:
portugal_salaries = portugal_data["salaries"]

In [13]:
print(portugal_salaries[0])

{'job': {'id': 'ACCOUNT-MANAGER', 'title': 'Account Manager'}, 'salary_percentiles': {'percentile_25': 13069.80634374441, 'percentile_50': 18701.600947355048, 'percentile_75': 26760.142330763127}}


In [14]:
for salary in portugal_salaries:
    if salary["job"]["id"] == "ARCHITECT":
        print(salary["salary_percentiles"])

{'percentile_25': 12024.33680305131, 'percentile_50': 14889.589438353738, 'percentile_75': 18437.5967900763}


In [15]:
def get_country_salaries(country_url, job_title):
    request = requests.get(country_url+ "salaries")
    if request.ok:
        salaries = request.json()["salaries"]
        for salary in salaries:
            if salary["job"]["id"] == job_title:
                return salary["salary_percentiles"]["percentile_50"]

In [16]:
architect_salaries = {}
for country_name, country_url in countries_dict.items():
    architect_salaries[country_name] = get_country_salaries(country_url, "ARCHITECT")

In [21]:
architect_salaries

{'Aland Islands': 70097.24168877421,
 'Albania': 7972.248496647717,
 'Andorra': 53978.46198233692,
 'Austria': 24033.926657851098,
 'Belarus': 9586.147151187122,
 'Belgium': 36315.91978741829,
 'Bosnia and Herzegovina': 9925.194490409021,
 'Bulgaria': 12490.58559274332,
 'Croatia': 17501.045135253615,
 'Cyprus': 20317.89158992561,
 'Czechia': 40080.24448281322,
 'Denmark': 77466.09424075867,
 'Estonia': 19127.914698175406,
 'Faroe Islands': 57742.680713716494,
 'Finland': 45734.124096882435,
 'France': 34987.72205196216,
 'Germany': 41262.04287660226,
 'Gibraltar': 91912.05924232888,
 'Greece': 15691.864515210455,
 'Guernsey': 69725.84555649548,
 'Hungary': 19216.42335653538,
 'Iceland': 104250.04055167825,
 'Ireland': 39341.01428404915,
 'Isle of Man': 73445.52996100018,
 'Italy': 17709.32474689936,
 'Jersey': 82675.66999749502,
 'Kosovo': 11210.80526619977,
 'Latvia': 16773.680681540845,
 'Liechtenstein': None,
 'Lithuania': 12715.863745737244,
 'Luxembourg': 88151.74876323169,
 'Mac

In [22]:
max_salary = 0
max_salary_country = ""
for country, median_salary in architect_salaries.items():
    if median_salary and median_salary> max_salary:
        max_salary_country = country
        max_salary = median_salary

In [23]:
print("The country with the highest 50% percentile salary is {} with {}$".format(
            max_salary_country, max_salary))

The country with the highest 50% percentile salary is Monaco with 147586.8475087603$
